Modified from https://gist.github.com/jkleint/1d878d0401b28b281eb75016ed29f2ee


Example of using Keras to implement a 1D convolutional neural network (CNN) for timeseries prediction.


In [1]:
from __future__ import print_function, division

import numpy as np
from keras.layers import Convolution1D, Dense, MaxPooling1D, Flatten
from keras.models import Sequential

Using TensorFlow backend.


In [2]:
def make_timeseries_regressor(window_size, filter_length, nb_input_series=1, nb_outputs=1, nb_filter=4):
    """:Return: a Keras Model for predicting the next value in a timeseries given a fixed-size lookback window of previous values.

    The model can handle multiple input timeseries (`nb_input_series`) and multiple prediction targets (`nb_outputs`).

    :param int window_size: The number of previous timeseries values to use as input features.  Also called lag or lookback.
    :param int nb_input_series: The number of input timeseries; 1 for a single timeseries.
      The `X` input to ``fit()`` should be an array of shape ``(n_instances, window_size, nb_input_series)``; each instance is
      a 2D array of shape ``(window_size, nb_input_series)``.  For example, for `window_size` = 3 and `nb_input_series` = 1 (a
      single timeseries), one instance could be ``[[0], [1], [2]]``. See ``make_timeseries_instances()``.
    :param int nb_outputs: The output dimension, often equal to the number of inputs.
      For each input instance (array with shape ``(window_size, nb_input_series)``), the output is a vector of size `nb_outputs`,
      usually the value(s) predicted to come after the last value in that input instance, i.e., the next value
      in the sequence. The `y` input to ``fit()`` should be an array of shape ``(n_instances, nb_outputs)``.
    :param int filter_length: the size (along the `window_size` dimension) of the sliding window that gets convolved with
      each position along each instance. The difference between 1D and 2D convolution is that a 1D filter's "height" is fixed
      to the number of input timeseries (its "width" being `filter_length`), and it can only slide along the window
      dimension.  This is useful as generally the input timeseries have no spatial/ordinal relationship, so it's not
      meaningful to look for patterns that are invariant with respect to subsets of the timeseries.
    :param int nb_filter: The number of different filters to learn (roughly, input patterns to recognize).
    """
    model = Sequential((
        # The first conv layer learns `nb_filter` filters (aka kernels), each of size ``(filter_length, nb_input_series)``.
        # Its output will have shape (None, window_size - filter_length + 1, nb_filter), i.e., for each position in
        # the input timeseries, the activation of each filter at that position.
        Convolution1D(nb_filter=nb_filter, filter_length=filter_length, activation='relu', input_shape=(window_size, nb_input_series)),
        MaxPooling1D(),     # Downsample the output of convolution by 2X.
        Convolution1D(nb_filter=nb_filter, filter_length=filter_length, activation='relu'),
        MaxPooling1D(),
        Flatten(),
        Dense(nb_outputs, activation='linear'),     # For binary classification, change the activation to 'sigmoid'
    ))
    model.compile(loss='mse', optimizer='adam', metrics=['mae'])
    # To perform (binary) classification instead:
    # model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['binary_accuracy'])
    return model

In [3]:
def make_timeseries_instances(timeseries, window_size):
    """Make input features and prediction targets from a `timeseries` for use in machine learning.

    :return: A tuple of `(X, y, q)`.  `X` are the inputs to a predictor, a 3D ndarray with shape
      ``(timeseries.shape[0] - window_size, window_size, timeseries.shape[1] or 1)``.  For each row of `X`, the
      corresponding row of `y` is the next value in the timeseries.  The `q` or query is the last instance, what you would use
      to predict a hypothetical next (unprovided) value in the `timeseries`.
    :param ndarray timeseries: Either a simple vector, or a matrix of shape ``(timestep, series_num)``, i.e., time is axis 0 (the
      row) and the series is axis 1 (the column).
    :param int window_size: The number of samples to use as input prediction features (also called the lag or lookback).
    """
    timeseries = np.asarray(timeseries)
    assert 0 < window_size < timeseries.shape[0]
    X = np.atleast_3d(np.array([timeseries[start:start + window_size] for start in range(0, timeseries.shape[0] - window_size)]))
    y = timeseries[window_size:]
    q = np.atleast_3d([timeseries[-window_size:]])
    return X, y, q

In [4]:
def evaluate_timeseries(timeseries, window_size):
    """Create a 1D CNN regressor to predict the next value in a `timeseries` using the preceding `window_size` elements
    as input features and evaluate its performance.

    :param ndarray timeseries: Timeseries data with time increasing down the rows (the leading dimension/axis).
    :param int window_size: The number of previous timeseries values to use to predict the next.
    """
    filter_length = 5
    nb_filter = 4
    timeseries = np.atleast_2d(timeseries)
    if timeseries.shape[0] == 1:
        timeseries = timeseries.T       # Convert 1D vectors to 2D column vectors

    nb_samples, nb_series = timeseries.shape
    print('\n\nTimeseries ({} samples by {} series):\n'.format(nb_samples, nb_series), timeseries)
    model = make_timeseries_regressor(window_size=window_size, filter_length=filter_length, nb_input_series=nb_series, nb_outputs=nb_series, nb_filter=nb_filter)
    print('\n\nModel with input size {}, output size {}, {} conv filters of length {}'.format(model.input_shape, model.output_shape, nb_filter, filter_length))
    model.summary()

    X, y, q = make_timeseries_instances(timeseries, window_size)
    print('\n\nInput features:', X, '\n\nOutput labels:', y, '\n\nQuery vector:', q, sep='\n')
    test_size = int(0.01 * nb_samples)           # In real life you'd want to use 0.2 - 0.5
    X_train, X_test, y_train, y_test = X[:-test_size], X[-test_size:], y[:-test_size], y[-test_size:]
    model.fit(X_train, y_train, nb_epoch=25, batch_size=2, validation_data=(X_test, y_test))

    pred = model.predict(X_test)
    print('\n\nactual', 'predicted', sep='\t')
    for actual, predicted in zip(y_test, pred.squeeze()):
        print(actual.squeeze(), predicted, sep='\t')
    print('next', model.predict(q).squeeze(), sep='\t')

In [5]:
def main():
    """Prepare input data, build model, evaluate."""
    np.set_printoptions(threshold=25)
    ts_length = 1000
    window_size = 50

    print('\nSimple single timeseries vector prediction')
    timeseries = np.arange(ts_length)                   # The timeseries f(t) = t
    evaluate_timeseries(timeseries, window_size)

    print('\nMultiple-input, multiple-output prediction')
    timeseries = np.array([np.arange(ts_length), -np.arange(ts_length)]).T      # The timeseries f(t) = [t, -t]
    evaluate_timeseries(timeseries, window_size)


All above functions are exexuted line by line below to check data changes

In [8]:
np.set_printoptions(threshold=25)
ts_length = 1000
window_size = 50

print('\nSimple single timeseries vector prediction')
timeseries = np.arange(ts_length)                   # The timeseries f(t) = t


Simple single timeseries vector prediction


In [9]:
timeseries

array([  0,   1,   2, ..., 997, 998, 999])

In [10]:
filter_length = 5
nb_filter = 4
timeseries = np.atleast_2d(timeseries)

In [11]:
timeseries

array([[  0,   1,   2, ..., 997, 998, 999]])

In [12]:
if timeseries.shape[0] == 1:
    timeseries = timeseries.T       # Convert 1D vectors to 2D column vectors

In [14]:
timeseries.shape

(1000, 1)

In [15]:
nb_samples, nb_series = timeseries.shape
print('\n\nTimeseries ({} samples by {} series):\n'.format(nb_samples, nb_series), timeseries)



Timeseries (1000 samples by 1 series):
 [[  0]
 [  1]
 [  2]
 ...
 [997]
 [998]
 [999]]


In [17]:
model = make_timeseries_regressor(window_size=window_size, filter_length=filter_length, nb_input_series=nb_series, nb_outputs=nb_series, nb_filter=nb_filter)
print('\n\nModel with input size {}, output size {}, {} conv filters of length {}'.format(model.input_shape, model.output_shape, nb_filter, filter_length))
model.summary()



Model with input size (None, 50, 1), output size (None, 1), 4 conv filters of length 5
Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_5 (Conv1D)            (None, 46, 4)             24        
_________________________________________________________________
max_pooling1d_5 (MaxPooling1 (None, 23, 4)             0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 19, 4)             84        
_________________________________________________________________
max_pooling1d_6 (MaxPooling1 (None, 9, 4)              0         
_________________________________________________________________
flatten_3 (Flatten)          (None, 36)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 1)                 37        
Total params: 145
Trainable par

In [18]:
timeseries = np.asarray(timeseries)

In [19]:
timeseries

array([[  0],
       [  1],
       [  2],
       ...,
       [997],
       [998],
       [999]])

In [20]:
timeseries.shape

(1000, 1)

In [21]:
assert 0 < window_size < timeseries.shape[0]

In [22]:
X = np.atleast_3d(np.array([timeseries[start:start + window_size] for start in range(0, timeseries.shape[0] - window_size)]))

In [23]:
X.shape

(950, 50, 1)

In [24]:
y = timeseries[window_size:]
q = np.atleast_3d([timeseries[-window_size:]])

In [25]:
y

array([[ 50],
       [ 51],
       [ 52],
       ...,
       [997],
       [998],
       [999]])

In [28]:
y.shape

(950, 1)

In [27]:
q.shape

(1, 50, 1)

In [29]:
q

array([[[950],
        [951],
        [952],
        ...,
        [997],
        [998],
        [999]]])

In [30]:
print('\n\nInput features:', X, '\n\nOutput labels:', y, '\n\nQuery vector:', q, sep='\n')
test_size = int(0.01 * nb_samples)           # In real life you'd want to use 0.2 - 0.5
X_train, X_test, y_train, y_test = X[:-test_size], X[-test_size:], y[:-test_size], y[-test_size:]
model.fit(X_train, y_train, nb_epoch=25, batch_size=2, validation_data=(X_test, y_test))




Input features:
[[[  0]
  [  1]
  [  2]
  ...
  [ 47]
  [ 48]
  [ 49]]

 [[  1]
  [  2]
  [  3]
  ...
  [ 48]
  [ 49]
  [ 50]]

 [[  2]
  [  3]
  [  4]
  ...
  [ 49]
  [ 50]
  [ 51]]

 ...

 [[947]
  [948]
  [949]
  ...
  [994]
  [995]
  [996]]

 [[948]
  [949]
  [950]
  ...
  [995]
  [996]
  [997]]

 [[949]
  [950]
  [951]
  ...
  [996]
  [997]
  [998]]]


Output labels:
[[ 50]
 [ 51]
 [ 52]
 ...
 [997]
 [998]
 [999]]


Query vector:
[[[950]
  [951]
  [952]
  ...
  [997]
  [998]
  [999]]]
C:\Users\xuesong.wang1\AppData\Local\Programs\Python\Python37\lib\site-packages\ipykernel_launcher.py:4: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  after removing the cwd from sys.path.
Train on 940 samples, validate on 10 samples
Epoch 1/25
940/940 [==============================] - 1s 1ms/step - loss: 35031.8746 - mae: 78.6222 - val_loss: 379.5303 - val_mae: 19.4811
Epoch 2/25
940/940 [==============================] - 1s 1ms/step - loss: 141.6836 - mae: 10.0206 - 

In [32]:
pred = model.predict(X_test)
print('\n\nactual', 'predicted', sep='\t')
for actual, predicted in zip(y_test, pred.squeeze()):
    print(actual.squeeze(), predicted, sep='\t')
print('next', model.predict(q).squeeze(), sep='\t')



actual	predicted
990	992.317
991	993.3203
992	994.32367
993	995.32697
994	996.33026
995	997.3336
996	998.3369
997	999.3403
998	1000.3437
999	1001.3469
next	1002.35034


In [33]:
print('\nMultiple-input, multiple-output prediction')
timeseries = np.array([np.arange(ts_length), -np.arange(ts_length)]).T      # The timeseries f(t) = [t, -t]
evaluate_timeseries(timeseries, window_size)


Multiple-input, multiple-output prediction


Timeseries (1000 samples by 2 series):
 [[   0    0]
 [   1   -1]
 [   2   -2]
 ...
 [ 997 -997]
 [ 998 -998]
 [ 999 -999]]


Model with input size (None, 50, 2), output size (None, 2), 4 conv filters of length 5
Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv1d_7 (Conv1D)            (None, 46, 4)             44        
_________________________________________________________________
max_pooling1d_7 (MaxPooling1 (None, 23, 4)             0         
_________________________________________________________________
conv1d_8 (Conv1D)            (None, 19, 4)             84        
_________________________________________________________________
max_pooling1d_8 (MaxPooling1 (None, 9, 4)              0         
_________________________________________________________________
flatten_4 (Flatten)          (None, 36)                0   